This chapter is a continuation from chap 1 and dives deep into creating more pythonic objects.

## Object Representations

Python has 2 standard ways of representing an object
1. `repr()` -> `__repr__`
2. `str()` -> `__str__`

There are 2 additional methods called `__bytes__` and `__format__`.

In [11]:
# inplement a vector2D class
from array import array
import math

class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octects[0])
        menv = memoryview(octects[1:]).cast(typecode)
        return cls(*memv)

In [2]:
# lets see it in action
v1 = Vector2d(3, 4)
print(v1.x, v1.y)

3.0 4.0


In [3]:
x, y = v1
x, y

(3.0, 4.0)

In [4]:
v1

Vector2d(3.0, 4.0)

In [5]:
v1_clone = eval(repr(v1))
v1_clone == v1, repr(v1)

(True, 'Vector2d(3.0, 4.0)')

In [6]:
print(v1)

(3.0, 4.0)


In [7]:
octects = bytes(v1)
octects

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [8]:
abs(v1)

5.0

In [11]:
bool(v1), bool(Vector2d(0, 0))

(True, False)

We have implemented all the basic methods but the one obvious operation that is missing is rebuilding a `Vector2d` from the binary representation. This is the `classmethod` that we have implemented.

`classmethods` are used to define methods that operates on the class and not on the instances. Its most commonly used for alternative constructors.

`staticmethod` changes the method so that it receives no special first argument. It is a plain function that happens to live in the class body.

In [1]:
# comparing behaviours of classmethod and staticmethod

class Demo:
    
    @classmethod
    def klassmeth(*args):
        return args
    
    @staticmethod
    def statmeth(*args):
        return args

In [2]:
Demo.klassmeth()

(__main__.Demo,)

In [3]:
Demo.klassmeth('spam')

(__main__.Demo, 'spam')

In [4]:
Demo.statmeth()

()

In [5]:
Demo.statmeth('spam')

('spam',)

## Formatted Displays

The `format()` built-in function and the `str.format()` method delegates the actual formatting to each type by calling their `.__format__(format_spec)` method. The `format_spec` is a formatting specfier

In [6]:
# examples of format_spec

brl = 1/2.43
brl

0.4115226337448559

In [7]:
format(brl, '0.4f')

'0.4115'

In [8]:
'1 BRL = {rate:0.2f} USD'.format(rate=brl)

'1 BRL = 0.41 USD'

## A Hashable Vector2d

Rt now our `Vector2d` object is not hashable so can't be put in a string.

In [12]:
v1 = Vector2d(3, 4)
hash(v1)

TypeError: unhashable type: 'Vector2d'

In [13]:
set([v1])

TypeError: unhashable type: 'Vector2d'

To make it hashable we have to implement the `__hash__`(`__eq__` also is required, but we have that). We also have to make instances immutable.

In [14]:
# inplement a vector2D class
from array import array
import math

class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
        
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
        
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    # hashes of individual attributes are joined with XOR
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octects[0])
        menv = memoryview(octects[1:]).cast(typecode)
        return cls(*memv)

In [15]:
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)
hash(v1), hash(v2)

(7, 384307168202284039)

In [16]:
set([v1, v2])

{Vector2d(3.0, 4.0), Vector2d(3.1, 4.2)}

## Private and "Protected" Attributes in Python

Python has no explicit ways to create private variables. Instead what is does is a process called 'name mangling'. If we want to create a private attribute we prefix the attribute with 2 '_' (sunderscores). Python stores the name in the instance `__dict__` prefixed with a leading underscore and the classname. So in the above example `__X` becomes `_Vecotor2d__X`. 

Note its about safety and not security.

In [17]:
v1 = Vector2d(3, 4)
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

## `__slots__` Class Attribute